# Model Training

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
## data ingestion
#import libraries and creating connection with Mongodb
import pymongo
import json
import pickle as pk

client = pymongo.MongoClient("localhost")
print(client)
#defining database and collection
db = client["Regression"]
collection = db["household_power_consumption_pickles"]

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


### Fetching pickled data

In [3]:
result = tuple(collection.find({},{"_id":0}))

### Reading scaler object

In [4]:
scaler = pk.loads(result[0].get("scaler_obj"))
scaler

StandardScaler()

### Reading train data

In [5]:
## fetching training data
train_data = result[1].get("Train_data")
x_train = pk.loads(train_data[0])
y_train = pk.loads(train_data[1])
print(x_train.shape)
print(y_train.shape)

(10050, 5)
(10050,)


In [6]:
x_train

array([[-0.84369277,  0.25634068,  0.09358889, -0.8444995 ,  0.46133945],
       [ 0.02105447,  0.32773141,  0.05625804,  0.00681677,  0.16868898],
       [-0.86818452, -1.08223554,  0.10920689, -0.88930562,  0.46133945],
       ...,
       [-0.79282529, -1.08223554,  0.10958782, -0.79969338,  0.75398993],
       [-0.76833353, -0.15415603,  0.22081852, -0.75488726,  0.16868898],
       [-0.85688064, -1.08223554, -0.0907798 , -0.8444995 , -0.1239615 ]])

### Reading test data

In [7]:
## fetching test data
test_data = result[2].get("Test_data")
x_test = pk.loads(test_data[0])
y_test = pk.loads(test_data[1])
print(x_test.shape)
print(y_test.shape)

(4950, 5)
(4950,)


## Linear Regression Model training

In [8]:
## creating regression model and training it
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(x_train,y_train)

LinearRegression()

In [9]:
## Coffiecient and intercept of model
print("Coeffiecients of model: ",linear_model.coef_)
print("intercept of model: ",linear_model.intercept_)

Coeffiecients of model:  [ 1.98029229e+01 -3.29935732e-01 -2.40234406e-01 -8.82437690e+00
 -1.08463251e-02]
intercept of model:  8.815621890547265


In [10]:
##standardization of test data
x_test = scaler.transform(x_test)
x_test

array([[-0.80789713,  0.14925458,  0.20596236, -0.79969338,  0.16868898],
       [ 0.05308215,  0.79177116,  0.14082384,  0.05162289,  0.75398993],
       [ 0.20380062, -0.20769908,  0.08178015,  0.18604125,  1.33929088],
       ...,
       [-0.64775876, -0.45756664,  0.09320796, -0.66527502, -0.41661198],
       [ 0.39219871, -0.56465274,  0.06768585,  0.36526573, -1.58721388],
       [-0.74760974,  0.0243208 ,  0.15644185, -0.75488726,  0.16868898]])

In [11]:
## testing of model
linear_pred = linear_model.predict(x_test)
linear_pred

array([-0.22686034,  9.10802135, 11.24412632, ...,  1.99183722,
       13.5463144 ,  0.62473675])

In [12]:
## checking accuracy by MSE,MAE,RMSE,R2 and ajusted R2 error
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
print("Mean absolute error: ",mean_absolute_error(y_test,linear_pred))
print("Mean square error: ",mean_squared_error(y_test,linear_pred))
print("r2 score: ",r2_score(y_test,linear_pred))

Mean absolute error:  4.163092312620457
Mean square error:  44.73822277535633
r2 score:  0.7167263971026698


In [13]:
#Adjusted R2 score
r2 = r2_score(y_test,linear_pred)
N = x_train.shape[0] + x_test.shape[0]
P = x_train.shape[1]
adjusted_r2 = (1 - ((1-r2)*(N-1)))/(N-P-1)
print("Adjusted r2: ", adjusted_r2)

Adjusted r2:  -0.28330137187255267


Model performance is very Bad as adjusted r2 is negative

In [14]:
linear_model.score(x_train,y_train)

0.717830399543542

### Linear Regression Model score is 71%